In [1]:
import torch
import torchvision
from torchvision import models
from torchvision import transforms

import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import os
import time
import pathlib
import sys

In [2]:
repo_path = pathlib.Path(os.getcwd())
base_dir = repo_path / "Neural Style Transfer/"
base_dir

PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer')

In [3]:
os.chdir(base_dir)

In [4]:
content_dir_path = base_dir / "content images/"
style_dir_path = base_dir / "style images/"
content_images_path = [
    content_dir_path / img 
    for img in os.listdir(content_dir_path) 
    if img.split(".")[-1] 
    in ["jpg", "jpeg", "png"]
]
style_images_path = [
    style_dir_path / img
    for img in os.listdir(style_dir_path)
    if img.split(".")[-1] 
    in ["jpg", "jpeg", "png"]
]

content_images_path, style_images_path

([PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/content images/The-Boy.jpeg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/content images/The-Guardian-2.jpg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/content images/The-Guardian.jpg')],
 [PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/style images/red-abstract.jpg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/style images/puzzled_women.jpg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/style images/emotional-joshua-miels.jpg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/style images/van-gogh.jpg')])

In [7]:

device = torch.device('mps')

def load_image(
    image_path: str,
    transform=None,
    max_size=None,
    shape=None,
    pad=False
):
    """Load an image and convert it to a torch tensor."""
    image = Image.open(image_path)

    if max_size:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale
        image = image.resize(size.astype(int), Image.Resampling.LANCZOS)

    if transform:
        image = transform(image)#.unsqueeze(0)

    if shape:
        image = image.resize(
            shape,
            Image.LANCZOS
        )
    if pad:
        _, h, w = image.size()
        max_hw = max(h, w)
        ph = (max_hw - h) / 2
        pw = (max_hw - w) / 2
        lwp = int(pw if pw % 1 == 0 else pw + 0.5)
        rwp = int(pw // 1)
        thp = int(ph if ph % 1 == 0 else ph + 0.5)
        bhp = int(ph // 1)
        image = transforms.functional.pad(
            image,
            padding=(lwp, thp, rwp, bhp),
            fill=0,
            padding_mode="constant"
        )

    return image.numpy()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))])

# content = load_image( content_images[1], transform, max_size=400, pad=True)
# style = load_image(config.style, transform, shape=[content.size(2), content.size(3)])
content_images = torch.Tensor([
    load_image(
        path_,
        transform,
        max_size=400,
        pad=True
    ) for path_ in content_images_path
])

style_image = load_image(
        style_images_path[-1],
        transform,
        max_size=400,
        pad=True
)

target_images = content_images.clone().requires_grad_(True)

content_images = content_images.to(device)
style_image = torch.Tensor(style_image).to(device)
target_images = target_images.to(device)


In [9]:
class VGGNet(nn.Module):
    def __init__(self):
        """Select conv1_1 ~ conv5_1 activation maps."""
        super(VGGNet, self).__init__()
        self.select = ['0', '5', '10', '19', '28']
        self.vgg = models.vgg19(pretrained=True).features

    def forward(self, x):
        """Extract multiple convolutional feature maps."""
        features = []
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.select:
                features.append(x)
        return features




In [12]:
target_images

/Users/vineetmahajan/.pyenv/versions/miniconda3-4.12/envs/torch_gpu/lib/python3.9/site-packages/torch/_tensor_str.py:103: UserWarning: The operator 'aten::bitwise_and.Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1664817727684/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(tensor_view, torch.isfinite(tensor_view) & tensor_view.ne(0))


tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
        

In [21]:
optimizer = torch.optim.Adam([target_images], lr=0.003, betas=[0.5, 0.999])
# backbone = VGGNet().to(device).eval()

# for step in range(1):
#     target_features = backbone(target_images)
#     content_features = backbone(content_images)
#     style_features = backbone(style_image)
    
    
#     n, _, h, w = target_features[-1].shape
    


ValueError: can't optimize a non-leaf Tensor

In [19]:
h

25